# EXPLORATION 7. 아이유팬이 좋아할 만한 다른 아티스트 찾기

---

## Dataset

In [1]:
import os
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import numpy as np

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings = ratings[['user_id', 'movie_id', 'count']]

In [5]:
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## EDA

In [7]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [8]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [9]:
# 가장 인기있는 영화 30개(인기순)
movie_data = pd.merge(ratings, movies)
movie_count = movie_data.groupby('title')['count'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

## Add ratings

In [10]:
my_favorite = ['Godfather, The (1972)' , 'Back to the Future (1985)' ,'Matrix, The (1999)' ,'Men in Black (1997)' ,'Jurassic Park (1993)']
favorite_movie_id = movies[movies['title'].isin(my_favorite)]
my_movie = pd.DataFrame({'user_id': [6041]*5, 'movie_id': favorite_movie_id['movie_id'], 'count':[5]*5})

if not ratings.isin({'user_id':[6041]})['user_id'].any():
    ratings = ratings.append(my_movie)
ratings.tail(10)

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
476,6041,480,5
847,6041,858,5
1250,6041,1270,5
1539,6041,1580,5
2502,6041,2571,5


In [11]:
favorite_movie_id

,movie_id,title,genre
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
847,858,"Godfather, The (1972)",Action|Crime|Drama
1250,1270,Back to the Future (1985),Comedy|Sci-Fi
1539,1580,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi
2502,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller


In [12]:
movie_data = pd.merge(ratings, movies, on='movie_id')
movie_data

,user_id,movie_id,count,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...
836478,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836479,5854,3026,4,Slaughterhouse (1987),Horror
836480,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836481,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary


## CSR matrix

In [13]:
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
476,6041,480,5
847,6041,858,5
1250,6041,1270,5
1539,6041,1580,5


In [14]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

In [15]:
num_user

6040

In [16]:
num_movie

3628

In [17]:
ratings.user_id

0          1
1          1
2          1
3          1
4          1
        ... 
476     6041
847     6041
1250    6041
1539    6041
2502    6041
Name: user_id, Length: 836483, dtype: int64

In [18]:
ratings.movie_id

0       1193
1        661
2        914
3       3408
4       2355
        ... 
476      480
847      858
1250    1270
1539    1580
2502    2571
Name: movie_id, Length: 836483, dtype: int64

In [28]:
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## Train

In [29]:
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [30]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [31]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [32]:
als_model.fit(csr_data_transpose)

In [40]:
my_vector, godfather_vector = als_model.user_factors[6041], als_model.item_factors[858]

In [41]:
my_vector

array([ 0.75035006, -0.49860355,  0.59534186,  0.0477808 ,  0.12469427,
        0.5824468 ,  1.0919695 , -0.5908663 , -0.29494345,  0.0578953 ,
        0.166457  ,  0.11666913, -0.7629893 , -0.1779293 ,  0.44854715,
       -0.5762628 , -0.7563232 , -0.6972777 ,  0.37251186,  0.14894277,
        0.47760344,  0.5258629 ,  0.42462078, -0.32681116, -0.02922969,
        0.9154519 , -0.16689415, -0.27018905, -0.13747472,  0.29484344,
       -0.1336557 ,  0.7693108 , -0.17002018, -0.7422458 , -0.9200032 ,
        0.08694491,  0.4681314 ,  0.58021295, -0.57215816, -0.5003212 ,
        0.3588585 , -0.56689215, -0.38545883,  0.15023197,  0.11277196,
       -0.21094123,  0.44487232,  0.19072957, -0.5607172 , -0.19179294,
        0.16280483,  0.31478018,  1.1189628 , -1.1205025 , -0.17486824,
       -0.782638  , -0.06910197, -0.1526821 ,  0.16086905,  0.82563925,
        0.737147  ,  0.24753349,  0.3800099 ,  0.9977372 ,  0.06301129,
       -0.26439285, -0.10830349,  1.035042  , -1.0880293 ,  0.09

In [42]:
godfather_vector

array([ 0.05985765, -0.01372723,  0.04429444,  0.01429889,  0.03080881,
       -0.01503598,  0.03878707,  0.01408573,  0.02057759,  0.00457484,
       -0.00816196, -0.02563019, -0.0425645 , -0.0102001 ,  0.01382642,
       -0.02219125, -0.0247666 , -0.04544543,  0.04289563,  0.00928458,
        0.00638019,  0.00548006,  0.00359358,  0.00169747,  0.00338007,
        0.00696554,  0.02268499,  0.03720165,  0.01870935,  0.02392538,
       -0.02355551,  0.01090179,  0.0042061 ,  0.02177642, -0.00466667,
        0.00591959,  0.00217996,  0.03282752,  0.00160465, -0.00337794,
        0.00889444,  0.00253196, -0.001026  , -0.02574297,  0.03674709,
       -0.0045134 ,  0.03573947,  0.04585135, -0.01064881, -0.03727816,
       -0.01620101,  0.01608805,  0.0021775 ,  0.00108833,  0.01850596,
        0.00538774, -0.00445224, -0.00480368, -0.02492749,  0.0297561 ,
        0.07098795,  0.01868587,  0.03153877,  0.03204146,  0.01819223,
        0.04863114,  0.00276844,  0.02435071, -0.02661153, -0.01

In [43]:
np.dot(my_vector, godfather_vector)

0.62299526

In [44]:
toystory_vector = als_model.item_factors[1]
np.dot(my_vector, toystory_vector)

0.12919995

## Similar Movie

In [53]:
favorite_movie = 'Godfather, The (1972)'
movie_id = movies[movies['title']=='Godfather, The (1972)']['movie_id']
similar_movie = als_model.similar_items(movie_id.values[0], N=15)
similar_movie

[(858, 0.24189797),
 (1221, 0.2349469),
 (2023, 0.1311299),
 (1953, 0.105369635),
 (1787, 0.08254059),
 (2887, 0.08188054),
 (2695, 0.081740566),
 (1213, 0.0817032),
 (3595, 0.080228694),
 (923, 0.0792953),
 (872, 0.07732083),
 (1207, 0.07406155),
 (624, 0.073954254),
 (1387, 0.06986658),
 (111, 0.06899316)]

In [61]:
movies[movies['movie_id'].isin([s[0] for s in similar_movie])]

,movie_id,title,genre
109,111,Taxi Driver (1976),Drama|Thriller
619,624,Condition Red (1995),Action|Drama|Thriller
847,858,"Godfather, The (1972)",Action|Crime|Drama
861,872,Aiqing wansui (1994),Drama
911,923,Citizen Kane (1941),Drama
1189,1207,To Kill a Mockingbird (1962),Drama
1195,1213,GoodFellas (1990),Crime|Drama
1203,1221,"Godfather: Part II, The (1974)",Action|Crime|Drama
1366,1387,Jaws (1975),Action|Horror
1728,1787,Paralyzing Fear: The Story of Polio in America...,Documentary


## Favorite Movie

In [56]:
user = 6041
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(589, 0.6278053),
 (1221, 0.5633624),
 (2916, 0.42419213),
 (110, 0.35272127),
 (1573, 0.3320789),
 (260, 0.3155406),
 (1097, 0.29259765),
 (457, 0.2854962),
 (2028, 0.2798354),
 (780, 0.2767408),
 (2023, 0.27195495),
 (1527, 0.2685769),
 (1240, 0.2618122),
 (1210, 0.24674135),
 (1196, 0.24358779),
 (1544, 0.23555496),
 (3175, 0.21955483),
 (356, 0.21676847),
 (2628, 0.21062228),
 (3101, 0.20523289)]

In [58]:
movies[movies['movie_id'].isin([m[0] for m in movie_recommended])]

,movie_id,title,genre
108,110,Braveheart (1995),Action|Drama|War
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
352,356,Forrest Gump (1994),Comedy|Romance|War
453,457,"Fugitive, The (1993)",Action|Thriller
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
770,780,Independence Day (ID4) (1996),Action|Sci-Fi|War
1081,1097,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1203,1221,"Godfather: Part II, The (1974)",Action|Crime|Drama


## Report

좋아하는 영화로 다음의 영화들을 선택하였습니다.  
Godfather, The  
Back to the Future  
Matrix  
Men in Black  
Jurassic Park  

<br>

Godfather에 대한 선호도를 측정한 결과, 0.62 정도가 측정되었고, 좋아하는 영화 목록에 없는 toystory에 대한 선호도를 측정한 결과, 0.12 정도가 측정되었습니다.

<br>

Godfather와 유사한 영화를 추천받아봤는데, Godfather의 장르인 Action|Crime|Drama 와 비슷한 장르의 영화를 추천받았습니다.

<br>

좋아하는 영화추천에서는 대부분의 장르가 Action이었습니다.초기에 목록에 넣은 영화들의 장르에 Action이 많이 포함되어 있어서 이런 결과를 얻은것 같습니다.